In [1]:
from cassandra.cluster import Cluster
import pandas as pd

In [2]:
def pandas_factory(colnames, rows):    
    df= pd.DataFrame(rows, columns=colnames)    
    return df

In [3]:
cluster = Cluster(contact_points=['localhost'], port=9042)
session = cluster.connect()
session.set_keyspace('hotel')
session.row_factory = pandas_factory
#session.default_fetch_size = 10000000 #needed for large queries, otherwise driver will do pagination. Default is 50000.

# Consulta 1

In [16]:
rows_e = session.execute('SELECT * FROM empleado')
df_empleado=rows_e._current_rows
df_empleado.sort_values(by=['numreg'])

,numreg,cods,incorporacion,nombre,sueldo
4,1,1.0,1996-10-23,Luisa Blanco Baroja,1000.0
6,2,2.0,1996-10-23,Fernando Serrano Vázquez,1000.0
12,3,3.0,2000-01-01,Manuel Pérez Calo,900.0
7,4,4.0,2000-01-01,Ana Troncoso Calvo,900.0
0,5,1.0,2002-09-13,Alba Troncoso Calvo,NaN
9,6,5.0,2002-09-13,Jorge Alonso Alonso,NaN
8,7,5.0,2010-01-01,Fernando Soaje Alvarez,NaN
5,8,5.0,2010-01-01,Rosa Luigi Paz,1000.0
10,9,6.0,2010-01-01,Rafael Fuertes Cabrera,1100.0
1,10,NaN,2010-01-01,Antonio Sancho Sancho,1000.0


# Consulta 2

In [28]:
rows_e = session.execute("SELECT * FROM servicio")
df_servicio=rows_e._current_rows
df_servicio

,cods,costeinterno,descripcion,numreg
0,5,50,restaurante,8
1,1,50,tintoreria,1
2,2,30,plancha,2
3,4,15,bar,4
4,6,25,floristeria,9
5,3,60,lavanderia,3


In [53]:
df_q2 = pd.merge(left=df_empleado, right=df_servicio, left_on='cods', right_on='cods')
display(df_q2)
print('Consulta solicitada')
display(df_q2.loc[df_q2['descripcion'].str.contains('restaurante')])

,numreg_x,cods,incorporacion,nombre,sueldo,costeinterno,descripcion,numreg_y
0,5,1.0,2002-09-13,Alba Troncoso Calvo,NaN,50,tintoreria,1
1,1,1.0,1996-10-23,Luisa Blanco Baroja,1000.0,50,tintoreria,1
2,8,5.0,2010-01-01,Rosa Luigi Paz,1000.0,50,restaurante,8
3,7,5.0,2010-01-01,Fernando Soaje Alvarez,NaN,50,restaurante,8
4,6,5.0,2002-09-13,Jorge Alonso Alonso,NaN,50,restaurante,8
5,2,2.0,1996-10-23,Fernando Serrano Vázquez,1000.0,30,plancha,2
6,4,4.0,2000-01-01,Ana Troncoso Calvo,900.0,15,bar,4
7,9,6.0,2010-01-01,Rafael Fuertes Cabrera,1100.0,25,floristeria,9
8,3,3.0,2000-01-01,Manuel Pérez Calo,900.0,60,lavanderia,3


Consulta solicitada


,numreg_x,cods,incorporacion,nombre,sueldo,costeinterno,descripcion,numreg_y
2,8,5.0,2010-01-01,Rosa Luigi Paz,1000.0,50,restaurante,8
3,7,5.0,2010-01-01,Fernando Soaje Alvarez,NaN,50,restaurante,8
4,6,5.0,2002-09-13,Jorge Alonso Alonso,NaN,50,restaurante,8
